# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Ejemplos de Aprendizaje Automático (Machine Learning): Logistic Regression** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

**Integrantes**:
- Lorena Ruelas Gaytán
- Yael Alejandro Rodríguez Barreto
- Ximena Isaac Horta
- Alberto Renteria Camacho

In [2]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MLSpark-Logistic-Regression") \
    .master("spark://e3b046ba856a:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/29 00:25:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Preparación de Datos

In [4]:
from team_name.spark_utils import SparkUtils

csv_path = "/home/jovyan/notebooks/data/heartDisease/framingham.csv"

# Define schema for the DataFrame
schema = SparkUtils.generate_schema([
    ("male", "float"),
    ("age", "float"),
    ("education", "float"),
    ("currentSmoker", "float"),
    ("cigsPerDay", "float"),
    ("BPMeds", "float"),
    ("prevalentStroke", "float"),
    ("prevalentHyp", "float"),
    ("diabetes", "float"),
    ("totChol", "float"),
    ("sysBP", "float"),
    ("diaBP", "float"),
    ("BMI", "float"),
    ("heartRate", "float"),
    ("glucose", "float"),
    ("TenYearCHD", "float")
])


# Convert list to a DataFrame
df = spark.read.csv(csv_path, header=True, schema=schema)
#df = spark.createDataFrame(data, schema)
clean_df = df.dropna(how="any")


clean_df.show()

+----+----+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+-----+---------+-------+----------+
|male| age|education|currentSmoker|cigsPerDay|BPMeds|prevalentStroke|prevalentHyp|diabetes|totChol|sysBP|diaBP|  BMI|heartRate|glucose|TenYearCHD|
+----+----+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+-----+---------+-------+----------+
| 1.0|39.0|      4.0|          0.0|       0.0|   0.0|            0.0|         0.0|     0.0|  195.0|106.0| 70.0|26.97|     80.0|   77.0|       0.0|
| 0.0|46.0|      2.0|          0.0|       0.0|   0.0|            0.0|         0.0|     0.0|  250.0|121.0| 81.0|28.73|     95.0|   76.0|       0.0|
| 1.0|48.0|      1.0|          1.0|      20.0|   0.0|            0.0|         0.0|     0.0|  245.0|127.5| 80.0|25.34|     75.0|   70.0|       0.0|
| 0.0|61.0|      3.0|          1.0|      30.0|   0.0|            0.0|         1.0|     0.0|  225.0|150.0| 95.0|28.58| 

### Assemble the features into a single vector column

In [5]:
from pyspark.ml.feature import VectorAssembler

df = clean_df.withColumnRenamed("TenYearCHD", "label")

assembler = VectorAssembler(inputCols=["male", "age", "education", "currentSmoker", "cigsPerDay", "BPMeds", "prevalentStroke", "prevalentHyp", "diabetes", "totChol", "sysBP", "diaBP", "BMI", "heartRate", "glucose"], outputCol="features")
data_with_features = assembler.transform(df).select("label", "features")

### Split the data into training and test sets 80% training data and 20% testing data

In [6]:
train_data, test_data = data_with_features.randomSplit([0.8, 0.2], seed=1234)

### Show the whole dataset

In [7]:
# Mostrar el dataset original
print("Original Dataset")
data_with_features.show()

# Mostrar el dataset de entrenamiento
print("Train set")
train_data.show()


Original Dataset


25/04/29 00:25:50 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[1.0,39.0,4.0,0.0...|
|  0.0|(15,[1,2,9,10,11,...|
|  0.0|[1.0,48.0,1.0,1.0...|
|  1.0|[0.0,61.0,3.0,1.0...|
|  0.0|[0.0,46.0,3.0,1.0...|
|  0.0|[0.0,43.0,2.0,0.0...|
|  1.0|(15,[1,2,9,10,11,...|
|  0.0|[0.0,45.0,2.0,1.0...|
|  0.0|[1.0,52.0,1.0,0.0...|
|  0.0|[1.0,43.0,1.0,1.0...|
|  0.0|(15,[1,2,9,10,11,...|
|  0.0|(15,[1,2,9,10,11,...|
|  0.0|[1.0,46.0,1.0,1.0...|
|  0.0|[0.0,41.0,3.0,0.0...|
|  1.0|[0.0,38.0,2.0,1.0...|
|  0.0|[1.0,48.0,3.0,1.0...|
|  1.0|[0.0,46.0,2.0,1.0...|
|  0.0|[0.0,38.0,2.0,1.0...|
|  0.0|[1.0,41.0,2.0,0.0...|
|  0.0|[0.0,42.0,2.0,1.0...|
+-----+--------------------+
only showing top 20 rows

Train set
+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(15,[1,2,9,10,11,...|
|  0.0|(15,[1,2,9,10,11,...|
|  0.0|(15,[1,2,9,10,11,...|
|  0.0|(15,[1,2,9,10,11,...|
|  0.0|(15,[1,2,9,10,11,...|
|  0.0|(15,[1,2,9,10,11,...|
|  0.0|

### Create a logistic regression model

In [8]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=10, regParam=0.01)


# TRAINNING

In [9]:
lr_model = lr.fit(train_data)

# Print coefficients
print("Coefficients: " + str(lr_model.coefficients))

# Display model summary
training_summary = lr_model.summary

25/04/29 00:26:29 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/04/29 00:26:29 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


Coefficients: [0.4452204514191804,0.05502023704581662,-0.005981957123458555,0.13288946634661816,0.014078054908460756,0.20381267334203113,0.7378452494096633,0.17579813995375004,0.1054281636684124,0.002653991815607116,0.014925248129863155,0.0009680262107608984,0.0023616849292589105,-0.00455493418838834,0.008181404726362473]


# PREDICTIONS

In [10]:
# Use the trained model to make predictions on the test data
predictions = lr_model.transform(test_data)

# Show predictions
predictions.select("features", "prediction", "probability").show(100)

+--------------------+----------+--------------------+
|            features|prediction|         probability|
+--------------------+----------+--------------------+
|(15,[1,2,9,10,11,...|       0.0|[0.97307997228396...|
|(15,[1,2,9,10,11,...|       0.0|[0.97544049852057...|
|(15,[1,2,9,10,11,...|       0.0|[0.97668310812167...|
|(15,[1,2,9,10,11,...|       0.0|[0.97511789633773...|
|(15,[1,2,9,10,11,...|       0.0|[0.96912089314642...|
|(15,[1,2,9,10,11,...|       0.0|[0.97189152114976...|
|(15,[1,2,9,10,11,...|       0.0|[0.97406166706312...|
|(15,[1,2,9,10,11,...|       0.0|[0.95604219650089...|
|(15,[1,2,9,10,11,...|       0.0|[0.97631574844674...|
|(15,[1,2,9,10,11,...|       0.0|[0.95635794078057...|
|(15,[1,2,9,10,11,...|       0.0|[0.95994128841722...|
|(15,[1,2,9,10,11,...|       0.0|[0.95692612277421...|
|(15,[1,2,9,10,11,...|       0.0|[0.95943391997440...|
|(15,[1,2,9,10,11,...|       0.0|[0.95972432389436...|
|(15,[1,2,9,10,11,...|       0.0|[0.97453960736890...|
|(15,[1,2,

In [11]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")
accuracy = evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})
precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})
f1 = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})

print(f"f1:{f1}")


f1:0.7942887623584284


In [13]:
sc.stop()